In [5]:
import json

with open("korean_frequency.json", encoding="utf-8") as f:
    data = json.load(f)
target_words = sorted({w for group in data.values() for w in group})

In [13]:
# naver news에서 어떤 단어가 사용되는 예문 수집
from bs4 import BeautifulSoup
import requests
import time

def fetch_sentences(word, max_pages=1):
    results = []
    for page in range(1, max_pages+1):
        url = f"https://search.naver.com/search.naver?where=news&query={word}&start={(page-1)*10+1}"
        headers = {"User-Agent": "Mozilla/5.0"}
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "lxml")
        for title in soup.select(".news_tit"):
            results.append(title.get("title"))
        time.sleep(0.3)
    return results


In [14]:
# 형태소 분석
from konlpy.tag import Okt

okt = Okt()

def preprocess_sentences(sentences):
    processed = []
    for s in sentences:
        words = okt.morphs(s, stem=True)
        processed.append(words)
    return processed

In [15]:
# 4. 말뭉치 수집 및 학습
all_sentences = []
for word in target_words[:100]:  # 테스트: 상위 100개 단어만
    sents = fetch_sentences(word, max_pages=2)
    all_sentences.extend(sents)

KeyboardInterrupt: 

In [16]:
print(f"총 수집된 문장 수: {len(all_sentences)}")
print(f"전처리된 문장 수: {len(tokenized)}")
print("예시 문장:", all_sentences[0] if all_sentences else "없음")
print("예시 토큰:", tokenized[0] if tokenized else "없음")

총 수집된 문장 수: 0


NameError: name 'tokenized' is not defined

In [11]:
# Word2Vec learning

from gensim.models import Word2Vec

model = Word2Vec(
    sentences=preprocess_sentences(all_sentences),
    vector_size=300,
    window=5,
    min_count=2,
    workers=4
)
model.wv.save("wiktionary5800_custom.kv")


RuntimeError: you must first build vocabulary before training the model